In [1]:
import tensorflow as tf
import numpy as np
import pickle 
import matplotlib.pyplot as plt
%matplotlib inline

# 导入数据

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
#mnist = input_data.read_data_sets('/data')
#读取本地的mnist数据集
mnist = input_data.read_data_sets(r'F:/Python/Xiaojiang/data/')

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting F:/Python/Xiaojiang/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting F:/Python/Xiaojiang/data/train-labels-idx1-ubyte.gz
Extracting F:/Python/Xiaojiang/data/t10k-images-idx3-ubyte.gz
Extracting F:/Python/Xiaojiang/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


# input

In [3]:
#  真实数据和噪音数据
def get_inputs(real_size,noise_size):
    real_img = tf.placeholder(tf.float32,[None,real_size])
    noise_img = tf.placeholder(tf.float32,[None,noise_size])
    return real_img,noise_img

#  生成器
* noise_img:产生的噪音输入
* n_units：隐层单元个数
* out_dim：输出的大小（28*28*1）

In [4]:
def get_generator(noise_img,n_units,out_dim,reuse=False,alpha=0.01):
    with tf.variable_scope("generator",reuse=reuse):
        #hidden layer
        hidden1 = tf.layers.dense(noise_img,n_units)
        # leaky ReLU
        hidden1 = tf.maximum(alpha * hidden1,hidden1)
        #dropout
        hidden1 = tf.layers.dropout(hidden1,rate=0.2)
        
        # logits & outputs
        logits = tf.layers.dense(hidden1,out_dim)
        outputs = tf.tanh(logits)
        
        return logits,outputs

#  判别器
* img:输入
* n_units：隐层单元个数
* reuse：由于要使用两次

In [5]:
def get_discriminator(img, n_units,reuse=False, alpha=0.01):
    with tf.variable_scope("discriminator",reuse=reuse):
        # hidden layer
        hidden1 = tf.layers.dense(noise_img, n_units)
        #leaky ReLU
        hidden1 = tf.maximum(alpha * hidden1, hidden1)
        
        #logits & outputs
        logits = tf.layers.dense(hidden1, 1)
        outputs = tf.sigmoid(logits)
        return logits,outputs

#  网络参数定义
* img_size:输入大小
* noise_size：噪音图像大小
* g_units：生成器隐层参数
* d_units:判别器隐层参数
* learning_rate:学习率

In [6]:
img_size = mnist.train.images[0].shape[0]
noise_size = 100
g_units = 128
d_units = 128
learning_rate = 0.001
alpha = 0.01

# 构建网络

In [7]:
tf.reset_default_graph()
real_img,noise_img = get_inputs(img_size,noise_size)
#generator
g_logits,g_outputs = get_generator(noise_img,g_units,img_size)
#discriminator
d_logits_real,d_outputs_real = get_discriminator(real_img,d_units)
d_logits_fake,d_outputs_fake = get_discriminator(g_outputs,d_units,reuse=True)

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.dropout instead.


In [8]:
# discriminator的Loss
#识别真实图片
d_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_real,
                                                                    labels=tf.ones_like(d_logits_real)))
#识别生成的图片
d_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake,
                                                                    labels=tf.zeros_like(d_logits_fake)))
#总体loss
d_loss = tf.add(d_loss_real,d_loss_fake)

# generator的loss

g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake,
                                                               labels=tf.ones_like(d_logits_fake)))

# 优化器

In [10]:
#train_vars = tf.trainable_variables()

#generator
g_vars = [var for var in tf.trainable_variables() if var.name.startswith('generator')]
# discriminator
d_vars = [var for var in tf.trainable_variables() if var.name.startswith('discriminator')]
# optimizer
print("d_loss:")
print(d_loss)
print("g_loss")
print(g_loss)
d_train_opt =tf.train.AdamOptimizer(learning_rate).minimize(d_loss, var_list=d_vars)
g_train_opt =tf.train.AdamOptimizer(learning_rate).minimize(g_loss, var_list=d_vars)

d_loss:
Tensor("Add:0", shape=(), dtype=float32)
g_loss
Tensor("Mean_2:0", shape=(), dtype=float32)


# 训练

In [11]:
# batch_size
batch_size = 64
epochs = 300
# 抽取样本数
n_sample = 25

#存储测试样例
samples = []
#存储loss
losses = []
# 保存生成器变量
saver = tf.train.Saver(var_list= g_vars)
#开始训练
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for e in range(epochs):
        for batch_i in range(mnist.train.num_examples//batch_size):
            batch = mnist.train.next_batch(batch_size)
            batch_images = batch[0].reshape((batch_size,784))
            # 对图像像素及逆行scale，这是因为tanh输出的结果介于(-1，1)，real和fake图片共享discriminator的参数
            batch_images = batch_images*2 - 1
            #genetator的输入噪声
            batch_noise = np.random.uniform(-1, 1, size=(batch_size,noise_size))
            #Run optimizers
            _ = sess.run(d_train_opt, feed_dict={real_img:batch_images, noise_img:batch_noise})
            _ = sess.run(g_train_opt, feed_dict={noise_img: batch_noise})
        # 每一轮结束计算loss
        train_loss_d = sess.run(d_loss,
                               feed_dict = {real_img:batch_images,
                                           noise_img:batch_noise})
        #real img loss
        train_loss_d_real = sess.run(d_loss_real,
                               feed_dict = {real_img:batch_images,
                                           noise_img:batch_noise})
        #fake img loss
        train_loss_d_fake = sess.run(d_loss_fake,
                               feed_dict = {real_img:batch_images,
                                           noise_img:batch_noise})
        #generator loss
        train_loss_g = sess.run(g_loss,
                               feed_dict = {real_img:batch_images,
                                           noise_img:batch_noise})
        print("Epoch{}/{}...".format(e+1,epochs),
             "判别器损失：{:.4f}(判别真实的：{:.4f} + 判别生成的：{:.4f})...".format(train_loss_d,train_loss_d_real,train_loss_d_fake),
                "生成器损失：{:.4f}".format(train_loss_g))
        losses.append((train_loss_d, train_loss_d_real, train_loss_d_fake, train_loss_g))

        # 保存样本
        sample_noise = np.random.uniform(-1, 1, size=(n_sample, noise_size))
        gen_samples = sess.run(get_generator(noise_img, g_units, img_size, reuse=True),
                              feed_dict={noise_img:sample_noise})
        samples.append(gen_samples)

        saver.save(sess, './checkpoint/generator.ckpt')
#保存到本地
with open('train_samples.pkl','wb') as f:
    pickle.dump(samples,f)

Epoch1/300... 判别器损失：1.4495(判别真实的：0.4795 + 判别生成的：0.9700)... 生成器损失：0.4795
Epoch2/300... 判别器损失：1.4583(判别真实的：0.4669 + 判别生成的：0.9914)... 生成器损失：0.4669
Epoch3/300... 判别器损失：1.4542(判别真实的：0.4700 + 判别生成的：0.9842)... 生成器损失：0.4700
Epoch4/300... 判别器损失：1.4600(判别真实的：0.4623 + 判别生成的：0.9978)... 生成器损失：0.4623
Epoch5/300... 判别器损失：1.4550(判别真实的：0.4697 + 判别生成的：0.9853)... 生成器损失：0.4697
Epoch6/300... 判别器损失：1.4536(判别真实的：0.4713 + 判别生成的：0.9822)... 生成器损失：0.4713
Epoch7/300... 判别器损失：1.4553(判别真实的：0.4676 + 判别生成的：0.9876)... 生成器损失：0.4676
Epoch8/300... 判别器损失：1.4540(判别真实的：0.4702 + 判别生成的：0.9838)... 生成器损失：0.4702
Epoch9/300... 判别器损失：1.4545(判别真实的：0.4705 + 判别生成的：0.9840)... 生成器损失：0.4705
Epoch10/300... 判别器损失：1.4563(判别真实的：0.4672 + 判别生成的：0.9891)... 生成器损失：0.4672
Epoch11/300... 判别器损失：1.4598(判别真实的：0.4626 + 判别生成的：0.9972)... 生成器损失：0.4626
Epoch12/300... 判别器损失：1.4537(判别真实的：0.4719 + 判别生成的：0.9818)... 生成器损失：0.4719
Epoch13/300... 判别器损失：1.4556(判别真实的：0.4663 + 判别生成的：0.9893)... 生成器损失：0.4663
Epoch14/300... 判别器损失：1.4615(判别真实的：0.4594 + 判别生成的：1.0021)... 

KeyboardInterrupt: 